# DSDM 



In [1]:
from IPython.display import display, Markdown as md
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy
import numpy as np
import random

import pandas as pd
import pathlib
from preprocess import preprocess_text

from sklearn.neighbors import LocalOutlierFactor

import torch
import torchhd as thd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F 
from tqdm import tqdm
from typing import List

In [2]:
# Set device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Vector dimension
dim = 2000 

cleanup = {}

In [3]:
def fix_seed():
    seed = 42
    print("[ Using Seed : ", seed, " ]")

    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    numpy.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
def load_data(path, bs=0, shuffle=False):
    """Load data from file path. """
    text = pathlib.Path(path).read_text(encoding='utf-8')
    return text.splitlines()


def compute_distances_gpu(X, Y):
    """Compute Euclidean distance."""
    return torch.sqrt(-2 * torch.mm(X,Y.T) +
                    torch.sum(torch.pow(Y, 2),dim=1) +
                    torch.sum(torch.pow(X, 2),dim=1).view(-1,1))

In [4]:
class SONN(nn.Module):
    def __init__(self, address_size, learning_rate_ema, learning_rate_update, temperature, normalize=False):
        super(SONN, self).__init__()
        self.address_size = address_size
        self.addresses = torch.zeros(1, address_size).to(device)

        self.normalize = normalize

        self.ema = 0
        self.learning_rate_ema = learning_rate_ema
        self.learning_rate_update = learning_rate_update

        self.temperature = temperature
        
    def reset(self):
        self.ema = 0
        self.addresses = torch.zeros(1, self.address_size).to(device)
        
    def retrieve(self, query_address):
        with torch.no_grad():
            retrieved_content = torch.tensor([]).to(device)

            cos = torch.nn.CosineSimilarity()
            # Calculate the cosine similarities.
            if self.normalize: 
                similarities = cos(self.addresses.sgn(), query_address.sgn())
            else:
                similarities = cos(self.addresses, query_address)
            # Cosine distance tensor
            distances = 1 - similarities

            # Calculate the softmin weights.
            softmin_weights = F.softmin(distances/self.temperature, dim=-1)

            # Weight the memory addresses with the softmin weights.
            weighted_addresses = torch.matmul(softmin_weights, self.addresses.to(device)).view(-1)

            # Pool the weighted memory addresses to create the output.
            retrieved_content = torch.sum(weighted_addresses.view(1, -1), 0)

        return retrieved_content   

    
    def save(self, query_address):
        cos = torch.nn.CosineSimilarity()
        # Calculate the cosine similarities.
        if self.normalize: 
            similarities = cos(self.addresses.sgn(), query_address.sgn())
        else:
            similarities = cos(self.addresses, query_address)

        # Calculate the cosine distances.
        distances = 1 - similarities
        # Get the minimum distance and the corresponding address index.  
        min_distance = torch.min(distances, dim=0)[0].item()
        
        # Check if the minimum distance is bigger than the adaptive threshold.
        if min_distance > self.ema: # If the minimum distance is bigger, create a new address.
            # Add a new entry to the address matrix/tensor equal to the target address.
            self.addresses = torch.cat((self.addresses, query_address.view(1, -1)))
        else: # If the minimum distance is smaller or equal, update the memory addresses.
            # Apply the softmin function to the distance tensor the get the softmin weights.
            softmin_weights = F.softmin(distances/self.temperature, dim=-1)
            # Update the memory address space.
            self.addresses += self.learning_rate_update * torch.mul(softmin_weights.view(-1, 1), query_address - self.addresses)
        
        # Update the adaptive threshold.
        self.ema = self.learning_rate_ema * self.ema + (1 - self.learning_rate_ema) * min_distance

        return

In [5]:
def generate_atomic_HVs_from_tokens_and_add_them_to_cleanup(tokens: List[str]) -> None:
    global cleanup, dim

    for token in tokens:
        # Check if the token has been encountered before by querying the cleanup memory.
        entry = cleanup.get(token)
        # If it hasn't, 
        if entry == None:
            # Generate a random HV representation for the token.
            atomic_HV = thd.MAPTensor.random(1, dim)[0]
            # Add the HV to the cleanup memory.
            cleanup[token] = atomic_HV
    
    return


def generate_chunk_representations_and_save_them_to_memory(memory, tokens, chunk_lengths=[], output=False):
    # "n" represents the no. of tokens in the sentence, which is also the max. no. of tokens 
    # that can be grouped to form a chunk.
    n = len(tokens)
    chunk_lengths = np.array(chunk_lengths, dtype=int)

    # Generate all possible chunks.
    if len(chunk_lengths) == 0:
        chunk_lengths = np.arange(1, n +  1)
    else:
        # Remove lengths which are bigger than the maximum chunk length.
        chunk_lengths = chunk_lengths[chunk_lengths <= n]
   
    for no_tokens in chunk_lengths:
        if output:
            print("no. of tokens: ", no_tokens)
        for i in range(n):
            if output:
                print("start index: ", i)
            # If there are not enough tokens left to construct a chunk comprised of "no_tokens", break. 
            if i + no_tokens > len(tokens):
                if output:
                    print("Not enough tokens left.")
                break 
            HC_representation = thd.MAPTensor.empty(1, dim)[0]

            # Construct HC representation.
            for j in range(no_tokens):
                if output:
                    print(tokens[i + j])
                HC_representation += cleanup[tokens[i + j]]

            # Save the chunk HC representation to memory.
            memory.save(HC_representation)

    return

def generate_query(tokens: list):
  n = len(tokens)
  HC_representation = thd.MAPTensor.empty(1, dim)

  for i in range(n):
    # The token hasn't been encountered before.
    if cleanup.get(tokens[i]) == None:
        # Generate an atomic HC for the unencountered token.
        atomic_HC = thd.MAPTensor.random(1, dim)[0]
        # Add the atomic HC to the cleanup memory.
        cleanup[tokens[i]] = atomic_HC
        # Add the atomic (i.e., superpose) HC to the chunk HC representation.
        HC_representation += atomic_HC
    # The token has been encountered before.
    else:
        HC_representation += cleanup[tokens[i]]

    return HC_representation

## Run experiment

In [6]:
# Load data.
lines_raw = load_data('../data/data.txt')

# Preprocess input. 
lines_tokens = []
for line_raw in lines_raw:
    if line_raw.rstrip():
        lines_tokens.append(preprocess_text(line_raw))


address_size = dim
learning_rate_ema = 0.13
learning_rate_update = 0.004
temperature = 2.3

# Create DSDM instance.
memory_unnormalized = SONN(address_size=address_size, learning_rate_ema=learning_rate_ema, learning_rate_update=learning_rate_update, temperature=temperature)
memory_normalized = SONN(address_size=address_size, learning_rate_ema=learning_rate_ema, learning_rate_update=learning_rate_update, temperature=temperature, normalize=True)

memories = {"normalized": memory_normalized, "unnormalized": memory_unnormalized}

# Flush cleanup memory.
cleanup = {}

# Train memory.
for sentence_tokens in lines_tokens:
    generate_atomic_HVs_from_tokens_and_add_them_to_cleanup(sentence_tokens)
    for _, memory in memories.items():
        generate_chunk_representations_and_save_them_to_memory(memory, sentence_tokens, chunk_lengths=[2, 3])


In [13]:
def get_concepts(memory, sentence):
  display(md(f"**Sentence**: {sentence}"))
  retrieved_content = memory.retrieve(generate_query(preprocess_text(sentence)))

  sims_df = pd.DataFrame(columns=['token', 'sim'])

  for key, item in cleanup.items():
    sims_df = pd.concat([sims_df, pd.DataFrame([{'token': key, 'sim': thd.cosine_similarity(item,  retrieved_content).item()}])])

  display(sims_df.sort_values('sim', ascending=False).reset_index(drop=True))
  return

for memory_type, memory in memories.items():
    display(md(f"### <ins>{memory_type.capitalize()}</ins>"))
    get_concepts(memory, "The red house.")
    get_concepts(memory, "The house.")
    get_concepts(memory, "House.")


### <ins>Normalized</ins>

**Sentence**: The red house.

,chunk,sim
0,the,0.643602
1,red,0.574333
2,house,0.360556
3,blue,0.305549
4,yellow,0.088265
5,green,0.064607


**Sentence**: The house.

,chunk,sim
0,the,0.643602
1,red,0.574333
2,house,0.360556
3,blue,0.305549
4,yellow,0.088265
5,green,0.064607


...

,chunk,sim
0,the,0.677186
1,house,0.503781
2,red,0.360327
3,blue,0.285576
4,yellow,0.202818
5,green,0.173281


**Sentence**: The house.

,chunk,sim
0,the,0.677186
1,house,0.503781
2,red,0.360327
3,blue,0.285576
4,yellow,0.202818
5,green,0.173281


**Sentence**: House.

,chunk,sim
0,house,0.629846
1,the,0.563269
2,red,0.352134
3,blue,0.297484
4,yellow,0.216424
5,green,0.175963


In [14]:
def print_memory_addresses(memory):
  print("Number of constructed addresses/abstract concepts: ", len(memory.addresses))


  for address in memory.addresses:
    sims_df = pd.DataFrame(columns=['token', 'sim'])
    for key, item in cleanup.items():
      sims_df = pd.concat([sims_df, pd.DataFrame([{'token': key, 'sim': thd.cosine_similarity(item,  address).item()}])])

    display(sims_df.sort_values('sim', ascending=False))

  return

for memory_type, memory in memories.items():
    display(md(f"### <ins>{memory_type.capitalize()}</ins>"))
    print_memory_addresses(memory)

### <ins>Normalized</ins>

Number of constructed addresses/abstract concepts:  18


,token,sim
0,house,0.782559
0,the,0.427431
0,red,0.370483
0,blue,0.183932
0,green,0.146675
0,yellow,0.118449


,token,sim
0,the,0.692903
0,red,0.692662
0,house,0.015816
0,green,0.013204
0,yellow,0.008608
0,blue,-0.002090


,token,sim
0,the,0.704527
0,green,0.702559
0,yellow,0.005492
0,blue,-0.003546
0,house,-0.006052
0,red,-0.008179


...

,token,sim
0,the,0.699636
0,yellow,0.697492
0,house,0.027024
0,blue,0.024304
0,green,0.014273
0,red,-0.001987


,token,sim
0,house,0.720260
0,yellow,0.717783
0,blue,0.041653
0,red,0.031436
0,green,0.008314
0,the,-0.014338


,token,sim
0,house,0.587016
0,blue,0.583560
0,the,0.571832
0,yellow,0.026454
0,red,-0.017457
0,green,-0.018320


,token,sim
0,house,0.588192
0,red,0.565347
0,the,0.559937
0,yellow,0.025697
0,blue,0.009336
0,green,-0.001152


,token,sim
0,red,0.692948
0,the,0.692693
0,green,0.011542
0,yellow,0.007241
0,house,0.006275
0,blue,-0.004319
